<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>

# Climate Change Satellite Image Classification Competition Model Submission Guide - keras

---
**About the Original Data:**<br>
*Data and Description accessed from [Tensorflow](https://www.tensorflow.org/datasets/catalog/bigearthnet)* <br>
The BigEarthNet is a new large-scale Sentinel-2 benchmark archive, consisting of 590,326 Sentinel-2 image patches. The image patch size on the ground is 1.2 x 1.2 km with variable image size depending on the channel resolution. This is a multi-label dataset with 43 imbalanced labels, which has been simplified to single labels with 3 categories for the purposes of this competition.

To construct the BigEarthNet, 125 Sentinel-2 tiles acquired between June 2017 and May 2018 over the 10 countries (Austria, Belgium, Finland, Ireland, Kosovo, Lithuania, Luxembourg, Portugal, Serbia, Switzerland) of Europe were initially selected. All the tiles were atmospherically corrected by the Sentinel-2 Level 2A product generation and formatting tool (sen2cor). Then, they were divided into 590,326 non-overlapping image patches. Each image patch was annotated by the multiple land-cover classes (i.e., multi-labels) that were provided from the CORINE Land Cover database of the year 2018 (CLC 2018).

Bands and pixel resolution in meters:

    B01: Coastal aerosol; 60m
    B02: Blue; 10m
    B03: Green; 10m
    B04: Red; 10m
    B05: Vegetation red edge; 20m
    B06: Vegetation red edge; 20m
    B07: Vegetation red edge; 20m
    B08: NIR; 10m
    B09: Water vapor; 60m
    B11: SWIR; 20m
    B12: SWIR; 20m
    B8A: Narrow NIR; 20m

License: Community Data License Agreement - Permissive, Version 1.0."

**Competition Data Specifics:**<br>
For the purpose of this competition, the original BigEarthNet dataset has been simplified to 20,000 images (15,000 training images and 5,000 test images) with 3 categories: "forest", "nonforest", and "snow_shadow_cloud", which contains images of snow and clouds. <br>
Each "image" is a folder with 12 satellite image layers, each of which pics up on different features. The example preprocessor uses just three layers: B02, B03, and B04, which contain the standard RGB layers used in ML models. However, you are free to use any combination of the satellite image layers. 

**Data Source:**<br>
Sumbul, G, Charfuelan, M, Demir, B and Markl, V. (2019). BigEarthNet: A Large-Scale Benchmark Archive For Remote Sensing Image Understanding. *Computing Research Repository (CoRR), abs/1902.06148.* https://www.tensorflow.org/datasets/catalog/bigearthnet




# Overview
---

Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data / Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Load Data

In [1]:
!python -V

Python 3.8.0


In [2]:
#install aimodelshare library
#! pip install aimodelshare-nightly

In [3]:
# Get competition data - May take a couple minutes due to size of data set
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/climate_competition_data-repository:latest') 

In [4]:
# Unzip Data - May take a couple minutes due to size of data set
import zipfile
with zipfile.ZipFile('climate_competition_data/climate_competition_data.zip', 'r') as zip_ref:
    zip_ref.extractall('competition_data')

##2.   Preprocess data / Write and Save Preprocessor function


In [5]:
# Set up for data preprocessing
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [6]:
# Here is a pre-designed preprocessor, but you could also build your own to prepare the data differently

def preprocessor(imageband_directory):
    """
    This function preprocesses reads in images, resizes them to a fixed shape and
    min/max transforms them before converting feature values to float32 numeric values
    required by onnx files.

    params:
        imageband_directory
            path to folder with 13 satellite image bands

    returns:
        X
            numpy array of preprocessed image data

    """

    import PIL
    import os
    import numpy as np
    import tensorflow_datasets as tfds

    def _load_tif(data):
        """Loads TIF file and returns as float32 numpy array."""
        img = tfds.core.lazy_imports.PIL_Image.open(data)
        img = np.array(img.getdata()).reshape(img.size).astype(np.float32)
        return img

    def preprocess_image(imgarray): 
        return imgarray
#         return (imgarray/np.max(imgarray))  # Preprocessing for Model-8 , Divide by max of each channel.
#         return (imgarray*255/np.max(imgarray))  # Preprocessing for Model-9 , Divide by max of each channel.

    image_list = []

    filelist1 = os.listdir(imageband_directory)

    for fpath in filelist1:
        fullpath = imageband_directory+"/"+fpath

        if fullpath.endswith(('B02.tif', 'B03.tif', 'B04.tif')):
            imgarray = _load_tif(imageband_directory+"/"+fpath)
            image_list.append(preprocess_image(imgarray))

        if fullpath.endswith('B05.tif'):
            vegetation_red_edge1 = _load_tif(imageband_directory+"/"+fpath)

        if fullpath.endswith('B06.tif'):
            vegetation_red_edge2 = _load_tif(imageband_directory+"/"+fpath)

        if fullpath.endswith('B07.tif'):
            vegetation_red_edge3 = _load_tif(imageband_directory+"/"+fpath)

        if fullpath.endswith('B11.tif'):
            SWIR_1 = _load_tif(imageband_directory+"/"+fpath)

        if fullpath.endswith('B12.tif'):
            SWIR_2 = _load_tif(imageband_directory+"/"+fpath)

        if fullpath.endswith('B8A.tif'):
            narrow_nir = _load_tif(imageband_directory+"/"+fpath)

        if fullpath.endswith('B08.tif'):
            nir = _load_tif(imageband_directory+"/"+fpath)

    mean_vegetation = (vegetation_red_edge1 + vegetation_red_edge2 + vegetation_red_edge3)/3

    mean_swir = (SWIR_1 + SWIR_2)/2

    mean_vegetation = np.repeat(np.repeat(mean_vegetation, 2, axis=1), 2, axis=0)
    
    mean_swir = np.repeat(np.repeat(mean_swir,2,axis=1),2,axis=0)

    narrow_nir = np.repeat(np.repeat(narrow_nir,2,axis=1),2,axis=0)

    mean_nir = (narrow_nir + nir)/2

    image_list.append(preprocess_image(mean_vegetation))
    image_list.append(preprocess_image(mean_swir))
    image_list.append(preprocess_image(mean_nir))

    X = np.stack(image_list, axis=2)   # to get (height,width,3)

    # Expand dims to add "1" to object shape [1, h, w, channels] for keras model.
    X = np.expand_dims(X, axis=0)

    X = np.array(X, dtype=np.float32)  # Final shape for onnx runtime.
    
    return X

In [7]:
# # Create complete list of file names
# forestfilenames = ["competition_data/trainingdata/forest/" +
#                    x for x in os.listdir("competition_data/trainingdata/forest")]
# nonforestfilenames = ["competition_data/trainingdata/nonforest/" +
#                       x for x in os.listdir("competition_data/trainingdata/nonforest")]
# otherfilenames = ["competition_data/trainingdata/other/" +
#                   x for x in os.listdir("competition_data/trainingdata/other")]

# filenames = forestfilenames+nonforestfilenames+otherfilenames

# # preprocess rbg images into 120,120,3 numpy ndarray
# preprocessed_image_data = []
# for i in filenames:
#     try:
#         preprocessed_image_data.append(preprocessor(i))
#     except Exception as e:
#         print(e)
#         pass

In [8]:
# np.save("6_Channel_PreProcessedImages_Model9",preprocessed_image_data)

In [9]:
preprocessed_image_data = np.load("6_Channel_PreProcessedImages_Model9.npy")

In [10]:
# Set up y data
from itertools import repeat
forest=repeat("forest",5000)
nonforest=repeat("nonforest",5000)
other=repeat("snow_shadow_cloud",5000)
ylist=list(forest)+list(nonforest)+list(other)

In [11]:
# Shuffle X and y data
from sklearn.utils import shuffle
X_train, y_train = shuffle(preprocessed_image_data, ylist, random_state=0)

In [12]:
X_train=np.vstack(X_train) # convert X from list to array

In [13]:
X_train.shape

for i in range(6):
    print(f"Max Value of {i} channel is {X_train[:,:,:,i].max()}")

Max Value of 0 channel is 18468.0
Max Value of 1 channel is 18552.0
Max Value of 2 channel is 18581.0
Max Value of 3 channel is 16522.0
Max Value of 4 channel is 15270.0
Max Value of 5 channel is 16198.0


In [14]:
def divide_channelbyMax(X_train):
    max_ValueArray = [] 
    for i in range(6): 
        max_ValueArray.append(X_train[:,:,:,i].max())
        X_train[:,:,:,i] = (X_train[:,:,:,i]*255)/max_ValueArray[-1]
    return X_train,max_ValueArray

In [15]:
X_train,max_ValueArray = divide_channelbyMax(X_train)

In [26]:
max_ValueArray

[18468.0, 18552.0, 18581.0, 16522.0, 15270.0, 16198.0]

In [16]:
X_train.shape

for i in range(6):
    print(f"Max Value of {i} channel is {X_train[:,:,:,i].min()}")

Max Value of 0 channel is 0.0
Max Value of 1 channel is 0.0
Max Value of 2 channel is 0.0
Max Value of 3 channel is 0.0
Max Value of 4 channel is 0.0
Max Value of 5 channel is 0.0


##3. Fit model on preprocessed data and save preprocessor function and model 


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow as tf
import pandas 


In [18]:
# Run using GPU
with tf.device('/device:GPU:0'): # "/GPU:0": Short-hand notation for the first GPU of your machine that is visible to TensorFlow.
    
    model = Sequential()
    model.add(ResNet50(include_top=False , weights=None ,input_shape=(120,120,6)))
    model.add(GlobalAveragePooling2D())
    
    model.add(Dense(2048, activation='relu')) 
    
    model.add(Dense(2048, activation='relu'))
    
    model.add(Dense(512, activation='relu')) 
    model.add(Dropout(0.2)) 

    model.add(Dense(256, activation='relu')) 
    model.add(Dropout(0.2)) 
    
    model.add(Dense(3, activation='softmax')) # Last fully-connected layer of 3 outputs (3 categories).

    model.summary()
    
    
    
    model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.SGD(1e-4),
              metrics=['accuracy'])
    
    
    
    def scheduler(epoch, lr):
        if epoch < 10 :
            return lr
        else:
            return lr * tf.math.exp(-0.1)
    
    Model_Checkpoint = tf.keras.callbacks.ModelCheckpoint("Model9",save_best_only=True)
    
    
    Learningrate_Scheduler  = tf.keras.callbacks.LearningRateScheduler(scheduler)
    

    model.fit(  X_train, 
                pandas.get_dummies(y_train),
                validation_split = .3, 
                epochs = 20,
                callbacks=[Model_Checkpoint,Learningrate_Scheduler])
    

    model.save("Model9.h5")


2022-03-24 01:07:19.362867: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 01:07:19.545279: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 01:07:19.545832: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 01:07:19.547054: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23597120  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 2048)              4196352   
                                                                 
 dense_1 (Dense)             (None, 2048)              4196352   
                                                                 
 dense_2 (Dense)             (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                        

2022-03-24 01:07:24.180315: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3628800000 exceeds 10% of free system memory.
2022-03-24 01:07:27.828901: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3628800000 exceeds 10% of free system memory.


Epoch 1/20


2022-03-24 01:07:35.298328: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-03-24 01:07:36.127938: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


...

INFO:tensorflow:Assets written to: Model9/assets


329/329 [==============================] - 79s 239ms/step - loss: 0.5995 - accuracy: 0.7582 - val_loss: 0.5733 - val_accuracy: 0.7604 - lr: 4.4933e-05
Epoch 19/20
329/329 [==============================] - 57s 175ms/step - loss: 0.5899 - accuracy: 0.7650 - val_loss: 0.5878 - val_accuracy: 0.7496 - lr: 4.0657e-05
Epoch 20/20
329/329 [==============================] - ETA: 0s - loss: 0.5833 - accuracy: 0.7662

INFO:tensorflow:Assets written to: Model9/assets


INFO:tensorflow:Assets written to: Model9/assets


329/329 [==============================] - 79s 240ms/step - loss: 0.5833 - accuracy: 0.7662 - val_loss: 0.5637 - val_accuracy: 0.7622 - lr: 3.6788e-05


In [19]:
print("Did it run ")


Did it run 


#### Save preprocessor function to local "preprocessor.zip" file

In [20]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

cannot pickle 'module' object
Your preprocessor is now saved to 'preprocessor.zip'


#### Save model to local ".onnx" file

In [21]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

/home/ecbm4040/climate_change/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/lib/python3.8/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-03-24 01:33:53,843 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2022-03-24 01:33:59,960 - INFO - Signatures found in model: [serving_default].
2022-03-24 01:33:59,960 - WARNING - '--signature_def' not specified, using first signature: serving_default
2022-03-24 01:33:59,962 - INFO - Output names: ['dense_4']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2022-03-24 01:34:03,561 - WARNING - From /home/ecbm4040/climate_change

## 4. Generate predictions from X_test data and submit model to competition


In [22]:
# import and preprocess X_test images in correct order...
# ...for leaderboard prediction submissions
filenumbers = [str(x) for x in range(1, 5001)]
filenames = ["competition_data/testdata/test/test"+x for x in filenumbers]

# preprocess rbg images into 120,120,3 numpy ndarray
preprocessed_image_data = []
for i in filenames:
    try:
        preprocessed_image_data.append(preprocessor(i))
    except:
        pass

In [23]:
X_test=np.vstack(preprocessed_image_data) 


In [24]:
X_test.shape

(5000, 120, 120, 6)

In [27]:
def divide_TestchannelbyMax(X_test,max_ValueArray):
    for i in range(6): 
        X_test[:,:,:,i] = (X_test[:,:,:,i]*255)/max_ValueArray[i]
    return X_test

X_train = divide_TestchannelbyMax(X_test,max_ValueArray)

In [29]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials

# Note -- This is the unique rest api that powers this climate change image classification  Model Plaground
# ... Update the apiurl if submitting to a new competition

apiurl="https://srdmat3yhf.execute-api.us-east-1.amazonaws.com/prod/m"
set_credentials(apiurl=apiurl)

AI Modelshare Username:········
AI Modelshare Password:········
AI Model Share login credentials set successfully.


In [30]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [31]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [pandas.get_dummies(y_train).columns[i] for i in prediction_column_index]

2022-03-24 01:38:54.661266: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1728000000 exceeds 10% of free system memory.


157/157 [==============================] - 8s 47ms/step


In [32]:
np.save("Predictions_Model9",prediction_labels)

In [33]:
# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

 15% [........                                            ]  1867776 / 11915122

100% [....................................................] 11915122 / 11915122

/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:287: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  leaderboard = leaderboard.append(metadata, ignore_index=True, sort=False)
/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:289: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  leaderboard['username']=leaderboard.pop("username")
/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:290: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=

100% [........................................................] 775866 / 775866

/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:287: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  leaderboard = leaderboard.append(metadata, ignore_index=True, sort=False)


Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 152

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1535


In [34]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

In [37]:
data[data["username"] == "vkalmath"]

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,...,swish_act,loss,optimizer,memory_size,Member1,Member2,team,username,timestamp,version
1,0.7648,0.776539,0.791213,0.827111,keras,NaN,True,Sequential,8.0,136387.0,...,NaN,function,Adam,4813664.0,NaN,NaN,NaN,vkalmath,2022-03-19 19:11:33.485952,88
22,0.7276,0.725968,0.722002,0.785111,keras,NaN,True,Sequential,7.0,136387.0,...,NaN,function,Adam,4803824.0,NaN,NaN,NaN,vkalmath,2022-03-18 19:48:55.382639,73
33,0.6964,0.698332,0.749256,0.724222,keras,NaN,True,Sequential,8.0,9573891.0,...,NaN,str,SGD,14168312.0,NaN,NaN,NaN,vkalmath,2022-03-24 01:39:59.262445,152
62,0.6840,0.641584,0.727566,0.649778,keras,True,True,Sequential,7.0,136387.0,...,NaN,function,Adam,4803824.0,NaN,NaN,NaN,vkalmath,2022-03-17 22:32:37.060508,64
63,0.6792,0.669706,0.672293,0.696000,keras,NaN,True,Sequential,8.0,9573891.0,...,NaN,str,SGD,14116664.0,NaN,NaN,NaN,vkalmath,2022-03-21 19:46:29.959795,120
67,0.6504,0.638267,0.715933,0.673333,keras,NaN,True,Sequential,11.0,5288963.0,...,NaN,function,Adam,4938160.0,NaN,NaN,NaN,vkalmath,2022-03-19 23:06:22.545450,91
75,0.6436,0.630026,0.718117,0.666000,keras,NaN,True,Sequential,11.0,5288963.0,...,NaN,function,Adam,4938160.0,NaN,NaN,NaN,vkalmath,2022-03-19 23:22:15.864357,92
150,0.3828,0.407672,0.461674,0.508222,keras,NaN,True,Sequential,7.0,9573891.0,...,NaN,str,SGD,14105328.0,NaN,NaN,NaN,vkalmath,2022-03-23 20:42:09.766975,145
162,0.3916,0.387116,0.427798,0.353556,keras,NaN,True,Sequential,8.0,1847811.0,...,NaN,str,RMSprop,2114800.0,NaN,NaN,NaN,vkalmath,2022-03-17 18:58:03.360045,59


## 5. Repeat submission process to improve place on leaderboard

*Train and submit your own models using code modeled after what you see above.*

It may also be useful to examine the architeture of models that perform particuarly well/poorly, or to compare models you've created with similar models submitted by others. Use the compare_models function in combination with the leaderboard to learn more about models that been previously submitted and potentially make decisiona about what you should do next.

In [ ]:
# Compare two or more models
data=mycompetition.compare_models([1, 5], verbose=1)
mycompetition.stylize_compare(data)